## Trains a torch based neural network with the MNIST Dataset and exports it to ONNX

#### for the used virtualenvironment run (on Linux) or look at ./create_venv_linux.sh

In [1]:
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import matplotlib
import pandas

### Define Neural Network 
Based on https://github.com/pytorch/examples/blob/main/mnist/main.py

In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch, dry_run=True, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### Load Dataset

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

train_dataset = datasets.MNIST('../data', train=True, download=True, transform = transform)
test_dataset = datasets.MNIST('../data', train=False, transform = transform)    

In [4]:
len(train_dataset)

60000

In [5]:
len(test_dataset)

10000

### Train

In [6]:
# if torch.cuda.is_available:
#    device = ("cuda")
# else:
device = ("cpu")
batch_size = 64
learning_rate = 1.0
gamma = 0.7
epochs = 15

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size)

In [8]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [9]:
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300351

Test set: Average loss: 2.2826, Accuracy: 2251/10000 (23%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.067795

Test set: Average loss: 2.0328, Accuracy: 3538/10000 (35%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.879444

Test set: Average loss: 1.8773, Accuracy: 3781/10000 (38%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.498722

Test set: Average loss: 1.6416, Accuracy: 4917/10000 (49%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 1.241429

Test set: Average loss: 1.5172, Accuracy: 5091/10000 (51%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 1.077310

Test set: Average loss: 1.4221, Accuracy: 5429/10000 (54%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 1.023719

Test set: Average loss: 1.3488, Accuracy: 5844/10000 (58%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.962732

Test set: Average loss: 1.3241, Accuracy: 5882/10000 (59%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.856613

Test set: Average loss: 1.2990, Accuracy: 5955/10000 (60%)

Train Epoch: 10 [0/60000 (0%)]	Loss: 0.778844


### Save

In [10]:


torch_input = torch.randn(1, 1, 28, 28)
torch_model = Net()
onnx_program = torch.onnx.export(torch_model, torch_input, "mnist_model.onnx", input_names=["input"], output_names=["output"])

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



### Load And Check

In [11]:
import onnx
onnx_model = onnx.load("mnist_model.onnx")
onnx.checker.check_model(onnx_model)

### you may visualize the model with https://netron.app/